In [15]:
from docx import Document
import re
from icecream import ic

In [16]:
def load_docx(file_path):
    try:
        return Document(file_path)
    except Exception as e:
        print(f"Error loading document: {e}")
        return None

doc = load_docx('data/vb_test.docx')
if not doc:
    exit()


In [17]:
import string
seps = '[.,;-+]'
ascii = string.ascii_lowercase
lowest_level = [(i + ")") for i in ascii]
levels = [lowest_level]

In [18]:
def extract_text(doc):
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                full_text.append(cell.text)
    return '\n'.join(full_text)

document_text = extract_text(doc)


In [19]:
alphabet_lst = [i for i in ascii]
digit_lst = [str(i) for i in range(99)]
bullet_levels1 = [["chương"], ["mục"], ["điều"], digit_lst, alphabet_lst]
bullet_levels2 = [["chương"], ["phụ lục"], digit_lst, alphabet_lst]

In [20]:
from collections import OrderedDict

In [21]:
text = []
tree = OrderedDict()
for i, para in enumerate(doc.paragraphs):
    text.append(para.text)

In [22]:
def check_in_first3(bullet, end_bullet_idx = 3):
    for i in range(end_bullet_idx):
        if  bullet in bullet_levels1[i]:
            return True
    return False

In [23]:
full_text = []
c_check = False
tracking = False
for i, para in enumerate(text):
    if c_check == True:
        c_check = False
        continue
    if para.split(" ")[0].lower() == "chương":
        para = para + " " + text[i + 1]
        c_check = True
    else:
        c_check = False
    first_token = para.split(" ")[0]
    bullet = "###"
    if len(first_token.strip()) > 0:
        bullet = re.split(r"[.,;)]",first_token)[0]
    if check_in_first3(bullet.lower()):
        tracking = True
    else:
        tracking = False
    if (not check_in_first3(bullet.lower())) and (not tracking):
        full_text[-1] = full_text[-1] + " > " + para
        continue
    full_text.append(para)

In [24]:
full_text

['Chương I NHỮNG QUY ĐỊNH CHUNG',
 'Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc. > 2. Nghị định này áp dụng đối với cơ quan, tổ chức, cá nhân trong nước và nước ngoài có hoạt động liên quan đến dược tại Việt Nam.',
 'Điều 2. Giải thích từ ngữ  > Trong Nghị định này các từ, ngữ dưới đây được hiểu như sau: > 1. Thông tin thuốc là việc thu thập, cung cấp các thông tin có liên quan đến thuốc bao gồm chỉ định, chống chỉ định, liều dùng, cách dùng, phản ứng có hại của thuốc và các thông tin khác liên quan đến chất lượng, an toàn, hiệu quả của thuốc do các cơ sở có tr

In [25]:
from collections import OrderedDict
def check_branch_level(tree):
    level = 0
    next_tree = tree
    # print(isinstance(next_tree, OrderedDict))  # Debugging statement
    while isinstance(next_tree, OrderedDict) and next_tree:  # Check if non-empty OrderedDict
        # Get the last key in the OrderedDict
        last_key = next(reversed(next_tree))
        next_tree = next_tree[last_key]
        level += 1
    return level

In [26]:
sample_tree = OrderedDict()
# sample_tree["a"] = OrderedDict()
print(sample_tree)
print(check_branch_level(sample_tree))

OrderedDict()
0


In [ ]:
def update_tree(bullet, par):
    for i in range (len(bullet_levels1)):
        if bullet.lower() in bullet_levels1[i]:
            k = tree
            current_branch_level = check_branch_level(k)
            if current_branch_level < i:
                val = OrderedDict()
                next_val = val
                for j in range (i - current_branch_level - 1):
                    next_val[""] = OrderedDict()
                    next_val = next_val[next(reversed(next_val))]
                next_val[par] = OrderedDict()
                while isinstance(k, OrderedDict) and k:  # Check if non-empty OrderedDict
                    # Get the last key in the OrderedDict
                    last_key = next(reversed(k))
                    k = k[last_key]
                k[""] = val
            else:
                for j in range(i):
                    k = k[next(reversed(k))]
                    print(k)
                k[par] = OrderedDict()

In [28]:
for i, para in enumerate(full_text):
    first_token = para.split(" ")[0]
    if len(first_token.strip()) > 0:
        bullet = re.split(r"[.,;)]",first_token)[0]
        update_tree(bullet, para)

OrderedDict([('', OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc. > 2. Nghị định này áp dụng đối với cơ quan, tổ chức, cá nhân trong nước và nước ngoài có hoạt động liên quan đến dược tại Việt Nam.', OrderedDict())]))])
OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý 

In [15]:
def flatten_tree(tree, parent_path="", separator=" > "):
    flat_list = []
    for key, value in tree.items():
        current_path = f"{parent_path}{separator}{key}" if parent_path else key
        # print(f"Processing: {current_path}")  # Debugging step
        if value:  # Check if the value is not blank OrderedDict
            flat_list.extend(flatten_tree(value, current_path, separator))
        else:
            # If value is a blank ordered dict, convert value to blank string
            flat_list.append((current_path, ""))
    return flat_list

In [16]:
flat_data = flatten_tree(tree)
chunks = [f"{path}: {text}" for path, text in flat_data]


In [17]:
chunks

['Chương I NHỮNG QUY ĐỊNH CHUNG >  > Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc. > 2. Nghị định này áp dụng đối với cơ quan, tổ chức, cá nhân trong nước và nước ngoài có hoạt động liên quan đến dược tại Việt Nam.: ',
 'Chương I NHỮNG QUY ĐỊNH CHUNG >  > Điều 2. Giải thích từ ngữ  > Trong Nghị định này các từ, ngữ dưới đây được hiểu như sau: > 1. Thông tin thuốc là việc thu thập, cung cấp các thông tin có liên quan đến thuốc bao gồm chỉ định, chống chỉ định, liều dùng, cách dùng, phản ứng có hại của thuốc và các thông tin khác liên quan đến chất lượng, an toàn,

In [18]:
# from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
# from typing import List, Dict, Tuple
# # Initialize the embedding model
# embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")

# # Embed and store in FAISS
# db = FAISS.from_texts(texts=chunks, embedding=embedding_model)

# # Save the FAISS vector database
# vector_db_path = "vectorstores/db_faiss"
# db.save_local(vector_db_path)

In [19]:
import re
import json
# from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from icecream import ic
import pickle
# Example raw data (replace with your data)
# chunks = [
#     "Chương I NHỮNG QUY ĐỊNH CHUNG > > Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng \n 1. Nghị định này quy định về...",
#     "Chương I NHỮNG QUY ĐỊNH CHUNG > > Điều 2. Giải thích từ ngữ \n Trong Nghị định này các từ, ngữ dưới đây được hiểu như...",
#     "Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG... > Điều 3...",
#     "Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 2. ĐÀO TẠO CẬP NHẬT KIẾN THỨC CHUYÊN MÔN VỀ DƯỢC > Điều 8..."
#     # Add more chunks here
# ]

# Preprocess chunks
def preprocess_chunks(chunks):
    """
    Process raw chunks into a structured format with chapter, section, article, and content.
    """
    processed_chunks = []
    for idx, chunk in enumerate(chunks):
        # Clean up extra whitespace
        chunk = chunk.strip()
        # ic(chunk)
        # Match hierarchical parts using regex
        match = re.match(r"(Chương [^\>]+) > (Mục [^\>]+)? > (Điều [^\.\>]+)\. (.+)", chunk)
        # ic(match)
        if match:
            chapter = match.group(1).strip()
            section = match.group(2).strip() if match.group(2) else None
            article = match.group(3).strip()
            content = match.group(4).strip()
            # ic(match.group(4).strip())
        # break

            # Combine hierarchical info with content for embedding
            combined_text = f"{chapter} > {section if section else ''} > {article}. {content}".strip()

            # Append structured data
            processed_chunks.append({
                "id": idx + 1,          # Unique ID for each chunk
                "chapter": chapter,     # Chapter name
                "section": section,     # Section name (if any)
                "article": article,     # Article name
                "content": content,     # Content text
                "text": combined_text   # Full text for embedding
            })
        else:
            # Handle unmatched chunks (log for review)
            print(f"Warning: Could not process chunk: {chunk}")
    return processed_chunks

# Preprocess the chunks
processed_chunks = preprocess_chunks(chunks)
# ic(processed_chunks)
# Initialize Sentence Transformer model
# embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# embedding_mod = GPT4AllEmbeddings(model_file="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# # Embed the processed chunks
texts = [chunk['text'] for chunk in processed_chunks]
# embeddings = embedding_model.encode(texts, convert_to_numpy=True)
# # embeddings = embedding_model.embed_documents(texts)  # Fix: Use embed_query instead of encode
# # embeddings = np.array(embeddings)
# # Create FAISS index
# dimension = embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension)  # L2 (Euclidean) distance
# index.add(embeddings)

# # Save FAISS index and metadata to local
# faiss.write_index(index, "faiss_index.bin")

# # Save metadata to a JSON or pickle file
# with open("metadata.pkl", "wb") as f:
#     pickle.dump(processed_chunks, f)

# print("FAISS index and metadata saved successfully!")

# # Load FAISS index and metadata
# def load_faiss_and_metadata(index_path, metadata_path):
#     index = faiss.read_index(index_path)
#     with open(metadata_path, "rb") as f:
#         metadata = pickle.load(f)
#     return index, metadata

# # Load index and metadata
# index, loaded_metadata = load_faiss_and_metadata("faiss_index.bin", "metadata.pkl")
# print("FAISS index and metadata loaded successfully!")

# # Query FAISS
# def query_faiss(query, top_k=5):
#     query_embedding = embedding_model.encode(query, convert_to_numpy=True)  # Fix: Use embed_query instead of encode
#     # query_embedding = np.array(query_embedding).reshape(1, -1)  
#     ic(query_embedding)
#     distances, indices = index.search(query_embedding, top_k)
#     results = []
#     for idx, dist in zip(indices[0], distances[0]):
#         if idx != -1:
#             metadata = loaded_metadata[idx]
#             metadata['score'] = dist
#             results.append(metadata)
#     return results

# # Test the query
# query_text = "Quy định chi tiết về hồ sơ đề nghị cấp Chứng chỉ hành nghề dược?"
# results = query_faiss(query_text, top_k=3)

# # Display the results
# for result in results:
#     print(result['text'])

In [20]:
# embeddings = embedding_model.embed_documents(texts)  # Fix: Use embed_query instead of encode
# embeddings = np.array(embeddings)

In [21]:
# dimension = embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension)  # L2 (Euclidean) distance
# index.add(embeddings)

# # Save FAISS index and metadata to local
# faiss.write_index(index, "faiss_index.bin")

# # Save metadata to a JSON or pickle file
# with open("metadata.pkl", "wb") as f:
#     pickle.dump(processed_chunks, f)

# print("FAISS index and metadata saved successfully!")

# # Load FAISS index and metadata
# def load_faiss_and_metadata(index_path, metadata_path):
#     index = faiss.read_index(index_path)
#     with open(metadata_path, "rb") as f:
#         metadata = pickle.load(f)
#     return index, metadata

# # Load index and metadata
# index, loaded_metadata = load_faiss_and_metadata("faiss_index.bin", "metadata.pkl")
# print("FAISS index and metadata loaded successfully!")

In [22]:
# def query_faiss(query, top_k=5):
#     # query_embedding = embedding_model.encode(query, convert_to_numpy=True)  # Fix: Use embed_query instead of encode
#     # query_embedding = np.array(query_embedding).reshape(1, -1)  
#     query_embedding = embedding_model.embed_query(query)  # Fix: Use embed_query instead of encode
#     query_embedding = np.array(query_embedding).reshape(1,-1)
#     ic(query_embedding.shape)
#     distances, indices = index.search(query_embedding, top_k)
#     results = []
#     for idx, dist in zip(indices[0], distances[0]):
#         if idx != -1:
#             metadata = loaded_metadata[idx]
#             metadata['score'] = dist
#             results.append(metadata)
#     return results

# # Test the query
# query_text = "Thời hạn cấp chứng chỉ hành nghề dược"
# results = query_faiss(query_text, top_k=30)


In [23]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from sklearn.preprocessing import MinMaxScaler

### 1️⃣ Load Sentence Transformer Model for Embeddings
model = SentenceTransformer('dangvantuan/vietnamese-document-embedding', trust_remote_code=True)

### 3️⃣ Dense Indexing (FAISS)
# Convert documents to embeddings
embeddings = model.encode(texts, convert_to_numpy=True)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance metric
index.add(embeddings)  # Add embeddings to FAISS

# Save FAISS index and metadata to local
faiss.write_index(index, "faiss_index.bin")

# Save metadata to a JSON or pickle file
with open("metadata.pkl", "wb") as f:
    pickle.dump(processed_chunks, f)

print("FAISS index and metadata saved successfully!")

# Load FAISS index and metadata
def load_faiss_and_metadata(index_path, metadata_path):
    index = faiss.read_index(index_path)
    with open(metadata_path, "rb") as f:
        metadata = pickle.load(f)
    return index, metadata

index, loaded_metadata = load_faiss_and_metadata("faiss_index.bin", "metadata.pkl")
print("FAISS index and metadata loaded successfully!")

### 4️⃣ Sparse Indexing (BM25)
# Tokenize documents for BM25
tokenized_docs = [doc.split() for doc in texts]
bm25 = BM25Okapi(tokenized_docs)  # BM25 index


### **🔍 5️⃣ Hybrid Search Query**
query = "Thời hạn cấp chứng chỉ hành nghề dược là bao nhiêu"
query_embedding = model.encode([query], convert_to_numpy=True)

# Dense Retrieval (FAISS)
D, I = index.search(query_embedding, k=3)  # Retrieve top-3 similar docs
dense_results = [texts[i] for i in I[0]]
dense_scores = D[0]

# Sparse Retrieval (BM25)
query_tokens = query.split()
bm25_scores = bm25.get_scores(query_tokens)
top_k_bm25 = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:3]
sparse_results = [texts[i] for i in top_k_bm25]
sparse_scores = [bm25_scores[i] for i in top_k_bm25]


scaler = MinMaxScaler()

# Normalize FAISS scores
dense_scores = np.array(dense_scores).reshape(-1, 1)
sparse_scores = np.array(sparse_scores).reshape(-1, 1)

dense_scores = scaler.fit_transform(dense_scores).flatten()
sparse_scores = scaler.fit_transform(sparse_scores).flatten()

# Weighted Hybrid Scoring (Adjust Weights as Needed)
hybrid_results = []
for i, doc in enumerate(dense_results):
    hybrid_results.append((0.6 * dense_scores[i], doc))  # 60% weight to Dense
     
for i, doc in enumerate(sparse_results):
    hybrid_results.append((0.4 * sparse_scores[i], doc))  # 40% weight to Sparse

# Sort by Final Score
hybrid_results.sort(reverse=True, key=lambda x: x[0])
final_passages = [doc for _, doc in hybrid_results]

ic(final_passages)


/home/phongnd/anaconda3/envs/llm2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ic| final_passages: ['Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 2. ĐÀO TẠO CẬP NHẬT KIẾN THỨC '
                     'CHUYÊN MÔN VỀ DƯỢC > Điều 9. Yêu cầu đối với cơ sở đào tạo, cập nhật kiến '
                     'thức chuyên môn về dược cho người hành nghề dược > 1. Cơ sở đào tạo, cập '
                     'nhật kiến thức chuyên môn về dược phải đáp ứng các yêu cầu sau: > a) Là một '
                     'trong các tổ chức sau: Cơ sở giáo dục nghề nghiệp có đào tạo chuyên ngành y, '
                     'dược; cơ sở giáo dục có đào tạo mã ngành thuộc khối ngành khoa học sức khỏe; '
                     'viện nghiên cứu có chức năng đào tạo chuyên ngành y, dược; cơ sở có chức '
                     'năng đào tạo nhân lực

FAISS index and metadata saved successfully!
FAISS index and metadata loaded successfully!


'
                     'Luật dược, người đề nghị cấp lại Chứng chỉ hành nghề dược không phải nộp '
                     'phí.:',
                     'Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 2. ĐÀO TẠO CẬP NHẬT KIẾN THỨC '
                     'CHUYÊN MÔN VỀ DƯỢC > Điều 8. Chương trình, nội dung, hình thức, phương pháp, '
                     'thời gian đào tạo cập nhật kiến thức chuyên môn về dược > Cơ sở đào tạo cập '
                     'nhật kiến thức chuyên môn về dược phải xây dựng chương trình đào tạo bao gồm '
                     'các nội dung chính sau đây:  > 1. Nội dung đào tạo bao gồm:  > a) Kiến thức '
                     'chuyên ngành; > b) Pháp luật và quản lý chuyên môn về dược;  > c) Kỹ năng và '
                     'các kỹ thuật trong hành nghề dược. > 2. Hình thức, phương pháp dạy, học thực '
                     'hành đánh giá kết quả đầu ra của người học thực hành phù hợp với chương '
                     'trình đào tạo cho từng môn học, đối tượng, trình độ đà

['Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 2. ĐÀO TẠO CẬP NHẬT KIẾN THỨC CHUYÊN MÔN VỀ DƯỢC > Điều 9. Yêu cầu đối với cơ sở đào tạo, cập nhật kiến thức chuyên môn về dược cho người hành nghề dược > 1. Cơ sở đào tạo, cập nhật kiến thức chuyên môn về dược phải đáp ứng các yêu cầu sau: > a) Là một trong các tổ chức sau: Cơ sở giáo dục nghề nghiệp có đào tạo chuyên ngành y, dược; cơ sở giáo dục có đào tạo mã ngành thuộc khối ngành khoa học sức khỏe; viện nghiên cứu có chức năng đào tạo chuyên ngành y, dược; cơ sở có chức năng đào tạo nhân lực y tế; các hội nghề nghiệp về dược; > b) Có chương trình đào tạo, cập nhật kiến thức chuyên môn về dược theo quy định tại Điều 8 của Nghị định này; > c) Có cơ sở vật chất trang thiết bị đáp ứng yêu cầu của chương trình đào tạo; > d) Có người hướng dẫn, báo cáo viên tham gia đào tạo, cập nhật kiến thức chuyên môn về dược (sau đây gọi tắt là người hướng dẫn) đáp ứng các yêu cầu sau: > - Người hướng dẫn về kiến thức chuyên ngành về dược phải có một trong 

In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from rank_bm25 import BM25Okapi
import torch
import faiss
import numpy as np
import pickle

class RAGDeepSeekQwen():
    def __init__(self, vector_db_path = "vectorstores/db_faiss", 
                 embedding_model_file = 'dangvantuan/vietnamese-document-embedding',
                 model_file = "AITeamVN/Vi-Qwen2-7B-RAG",
                 ):
        
        self.vector_db_path = vector_db_path
        self.model_file = model_file
        # Initialize the embedding model
        self.embedding_model = SentenceTransformer(embedding_model_file, trust_remote_code=True)
        self.index, self.loaded_metadata = self.load_faiss_and_metadata("faiss_index.bin", "metadata.pkl")
        self.texts = [meta_data['text'] for meta_data in self.loaded_metadata]
        self.tokenized_docs = [doc.split() for doc in self.texts]

        # Load the FAISS vector database with the embedding model
        # self.db = FAISS.load_local(folder_path=vector_db_path, embeddings=self.embedding_model, allow_dangerous_deserialization = True)


        self.system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể."
        self.template = '''Chú ý các yêu cầu sau:
        - Câu trả lời phải chính xác và đầy đủ nếu ngữ cảnh có câu trả lời. 
        - Chỉ sử dụng các thông tin có trong ngữ cảnh được cung cấp.
        - Chỉ cần từ chối trả lời và không suy luận gì thêm nếu ngữ cảnh không có câu trả lời.
        - Nếu nhiều nội dung được lấy từ cùng 1 khoản trong tài liệu đã cho, trả về toàn bộ nội dung trong khoản đó một cách chính xác nhất, không thực hiện tóm tắt lại.
        Hãy trả lời câu hỏi dựa trên ngữ cảnh:
        ### Ngữ cảnh :
        {context}

        ### Câu hỏi :
        {question}

        ### Trả lời :'''

        # Khởi tạo mô hình LLM và tokenizer
        self.model, self.tokenizer = self.load_huggingface_model(self.model_file)
    def load_faiss_and_metadata(self, index_path, metadata_path):
        index = faiss.read_index(index_path)
        with open(metadata_path, "rb") as f:
            metadata = pickle.load(f)
        return index, metadata
    def search_query(self, query: str):
        """
        Perform a similarity search on the vector database.
        
        :param query: The query string.
        :param k: The number of top results to return.
        :return: The top results as a list of strings.
        """
        query_embedding = self.embedding_model.encode([query], convert_to_numpy=True)
        bm25 = BM25Okapi(self.tokenized_docs)
        
        # Dense Retrieval (FAISS)
        D, I = self.index.search(query_embedding, k=3)  # Retrieve top-3 similar docs
        dense_results = [self.texts[i] for i in I[0]]
        dense_scores = D[0]

        # Sparse Retrieval (BM25)
        query_tokens = query.split()
        bm25_scores = bm25.get_scores(query_tokens)
        top_k_bm25 = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:3]
        sparse_results = [self.texts[i] for i in top_k_bm25]
        sparse_scores = [bm25_scores[i] for i in top_k_bm25]


        scaler = MinMaxScaler()

        # Normalize FAISS scores
        dense_scores = np.array(dense_scores).reshape(-1, 1)
        sparse_scores = np.array(sparse_scores).reshape(-1, 1)

        dense_scores = scaler.fit_transform(dense_scores).flatten()
        sparse_scores = scaler.fit_transform(sparse_scores).flatten()

        # Weighted Hybrid Scoring (Adjust Weights as Needed)
        hybrid_results = []
        for i, doc in enumerate(dense_results):
            hybrid_results.append((0.6 * dense_scores[i], doc))  # 60% weight to Dense
            
        for i, doc in enumerate(sparse_results):
            hybrid_results.append((0.4 * sparse_scores[i], doc))  # 40% weight to Sparse

        # Sort by Final Score
        hybrid_results.sort(reverse=True, key=lambda x: x[0])
        final_passages = [doc for _, doc in hybrid_results]
        return final_passages
    
    def load_huggingface_model(self,model_file):
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,  # Tải trọng số được lượng hóa trước theo định dạng 4 bit
            bnb_4bit_quant_type="nf4",  # Sử dụng loại lượng hóa "nf4" cho trọng số 4 bit
            bnb_4bit_compute_dtype=torch.bfloat16,  # Sử dụng torch.bfloat16 cho các phép tính trung gian
            bnb_4bit_use_double_quant=True,  # Sử dụng độ chính xác kép để lượng hóa kích hoạt
        )
        # quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold = 6.0)
        model = AutoModelForCausalLM.from_pretrained(model_file, device_map="auto", quantization_config=quantization_config,)
        tokenizer = AutoTokenizer.from_pretrained(model_file)
        return model, tokenizer

    # Read the vector database (FAISS)
    def read_vectors_db(self):
        embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")
        db = FAISS.load_local(self.vector_db_path, embedding_model, allow_dangerous_deserialization=True)
        return db

    # Perform similarity search on the vector database
    def search_vector_db(self,query, k=2):
        db = self.read_vectors_db()
        results = db.similarity_search(query, k=k)
        return [result.page_content for result in results]
    
    def rag_answer(self, prompt):
        context_list = self.search_query(prompt)
        context = "\n".join(context_list)
        print("\n\n\nCONTEXT:", context)
        print("\n\n")
        conversation = [{"role": "system", "content": self.system_prompt }]
        conversation.append({"role": "user", "content": self.template.format(context = context, question = prompt)})
        with torch.inference_mode():
            text = self.tokenizer.apply_chat_template(
                conversation,
                tokenize=False,
                add_generation_prompt=True)
            model_inputs = self.tokenizer(text,return_tensors="pt").to(self.model.device)

            generated_ids = self.model.generate(
                model_inputs.input_ids,
                max_new_tokens=2048,
                temperature = 0.1,
                top_p=0.95,
                top_k=40,
            )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            return response
        
model = RAGDeepSeekQwen()
answer = model.rag_answer("Thời hạn cấp chứng chỉ hành nghề dược nằm trong điều nào của luật")
print(answer)

/home/phongnd/anaconda3/envs/llm2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.25s/it]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.





CONTEXT: Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 4. Quy định chi tiết về hồ sơ đề nghị cấp lại Chứng chỉ hành nghề dược > 1. Hồ sơ đề nghị cấp lại Chứng chỉ hành nghề dược thực hiện theo quy định tại Điều 25 của Luật dược và được quy định cụ thể như sau: > a) Đơn đề nghị cấp lại Chứng chỉ hành nghề dược thực hiện theo Mẫu số 04 tại Phụ lục I ban hành kèm theo Nghị định này, 02 ảnh chân dung cỡ 4 cm x 6 cm của người đề nghị cấp Chứng chỉ hành nghề dược chụp trên nền trắng trong thời gian không quá 06 tháng; > b) Bản sao Chứng chỉ hành nghề dược đã được cấp trừ trường hợp bị mất Chứng chỉ hành nghề dược.  > 2. Số lượng hồ sơ quy định tại Điều này là 01 bộ. :
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược > 1. Ngườ